# Harm Detection ML Model
Train a high-accuracy XGBoost model for classifying online text as Hate Speech, Offensive, or Neutral.

In [37]:

# 1. Load Libraries
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import pickle


In [38]:

# 2. Load Dataset
df = pd.read_csv("labeled_data.csv")[["tweet", "class"]]
df = df.rename(columns={"tweet": "text", "class": "label"})
df.head()


,text,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [39]:

# 3. Text Cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["text"] = df["text"].apply(clean_text)
df.head()


,text,label
0,rt mayasolovely as a woman you shouldnt compla...,2
1,rt mleew boy dats coldtyga dwn bad for cuffin ...,1
2,rt urkindofbrand dawg rt sbabylife you ever fu...,1
3,rt cganderson vivabased she look like a tranny,1
4,rt shenikaroberts the shit you hear about me m...,1


In [40]:

# 4. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, stratify=df["label"], random_state=42
)


In [41]:

# 5. Vectorization (TF-IDF with bigrams)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [42]:

# 6. Model Training (XGBoost Classifier)
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train_vec, y_train)


c:\Users\krupa\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:34:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [43]:

# 7. Evaluation
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred, target_names=["Hate Speech", "Offensive", "Neutral"]))


              precision    recall  f1-score   support

 Hate Speech       0.54      0.21      0.30       286
   Offensive       0.94      0.95      0.94      3838
     Neutral       0.80      0.94      0.87       833

    accuracy                           0.90      4957
   macro avg       0.76      0.70      0.70      4957
weighted avg       0.89      0.90      0.89      4957



In [44]:

# 8. Save Model and Vectorizer
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("✅ Model and vectorizer saved!")


✅ Model and vectorizer saved!
